In [43]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_PROJECT"] = ""   


In [48]:
from youtube_transcript_api import YouTubeTranscriptApi
from dotenv import load_dotenv

from youtube_transcript_api._errors import TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.prompts import PromptTemplate 


In [45]:
video_id="LCEmiRjPEtQ"
try:
  transcript_list= YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])
  transcript =" ".join(chunk["text"] for chunk in transcript_list)
  print(transcript)
 
except TranscriptsDisabled:
   print("no caption")


  

Please welcome former director of AI Tesla Andre Carpathy. [Music] Hello. [Music] Wow, a lot of people here. Hello. Um, okay. Yeah. So I'm excited to be here today to talk to you about software in the era of AI. And I'm told that many of you are students like bachelors, masters, PhD and so on. And you're about to enter the industry. And I think it's actually like an extremely unique and very interesting time to enter the industry right now. And I think fundamentally the reason for that is that um software is changing uh again. And I say again because I actually gave this talk already. Um but the problem is that software keeps changing. So I actually have a lot of material to create new talks and I think it's changing quite fundamentally. I think roughly speaking software has not changed much on such a fundamental level for 70 years. And then it's changed I think about twice quite rapidly in the last few years. And so there's just a huge amount of work to do a huge amount of software to

In [26]:
transcript_list

[{'text': 'Please welcome former director of AI',
  'start': 1.12,
  'duration': 6.05},
 {'text': 'Tesla Andre Carpathy.', 'start': 4.0, 'duration': 7.44},
 {'text': '[Music]', 'start': 7.17, 'duration': 7.27},
 {'text': 'Hello.', 'start': 11.44, 'duration': 3.0},
 {'text': '[Music]', 'start': 14.77, 'duration': 3.08},
 {'text': 'Wow, a lot of people here. Hello.',
  'start': 19.039,
  'duration': 5.761},
 {'text': "Um, okay. Yeah. So I'm excited to be",
  'start': 22.8,
  'duration': 4.399},
 {'text': 'here today to talk to you about software',
  'start': 24.8,
  'duration': 5.76},
 {'text': "in the era of AI. And I'm told that many",
  'start': 27.199,
  'duration': 5.36},
 {'text': 'of you are students like bachelors,',
  'start': 30.56,
  'duration': 3.839},
 {'text': "masters, PhD and so on. And you're about",
  'start': 32.559,
  'duration': 3.84},
 {'text': "to enter the industry. And I think it's",
  'start': 34.399,
  'duration': 3.361},
 {'text': 'actually like an extremely u

In [61]:
splitter= RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
chunks = splitter.create_documents([transcript])

In [62]:
len(chunks)

58

In [63]:
embeddings= OpenAIEmbeddings(model="text-embedding-3-small")
vector_store= FAISS.from_documents(chunks,embeddings)


In [55]:
vector_store.index_to_docstore_id

{0: 'e2342649-cbf1-4ef0-861e-e1aaf50aca49',
 1: '3f19dbef-a3f4-4833-8e53-2f958e0ddb39',
 2: '0c813f7a-5c3c-4ab5-82ad-c65bb5f4a33e',
 3: 'bf275055-fafa-4140-89b6-87e033779dd6',
 4: '23e79d0f-767a-4255-9bfb-b8f988bccc4d',
 5: '2b3c6a12-4acf-40f8-972d-5f60c609b87f',
 6: '5940d0b3-5d97-4272-acaf-9b481fd61596',
 7: 'bbde9bfa-570b-4caf-989f-f18903ff27da',
 8: '618a9429-b813-4647-ac2c-fa3fe15dc469',
 9: '3d6f4970-61a0-4be6-94d4-596eb7721c9e',
 10: '7f6d5b4e-9ec3-4c7c-af4c-e62f34eb4f28',
 11: '0c8a3059-3cea-4ff1-8288-03589eba4f86',
 12: 'c03956b5-eedb-4d44-a64f-7b3ebf122d69',
 13: '9515dc0e-03f5-4dac-a4d6-933c21440a8e',
 14: '68f5ccf2-cb3a-4a52-8304-d289b971143c',
 15: '83ace61c-7c79-4440-9e1b-ffe300d57795',
 16: '580d1d12-072e-4659-84a7-351f50eea03e',
 17: '87dd8b9d-3b90-422b-b50d-c0414d72886f',
 18: '4b7244f7-a0c6-4eb9-a74a-c47adf88f5f9',
 19: 'a32fa6e0-2617-4ac7-90da-852a7e040e17',
 20: 'eff79399-4d3e-45fc-afbf-7fb71a66facf',
 21: '5ae7ea2a-e892-4b4f-a93a-4eeb58a47638',
 22: '9ea2294a-f508-

In [64]:
retriever= vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [86]:
retrived_doc=retriever.invoke('what is vibe coding?')

In [87]:
llm= ChatOpenAI(model="gpt-4o-mini",temperature=0.2)



In [95]:
prompt= PromptTemplate(
    template="""Answer ONLY from the provided transcript context.
    If the context is insuffcient, just say you don't know .\n{context}\n question:{question}""",
    input_variables=['context','questions']
)

In [96]:
content_text="\n".join(doc.page_content for doc in retrived_doc)

In [90]:
content_text

"I think is very unique. Not only is there a new type of programming language that allows for autonomy in software but also as I mentioned it's programmed in English which is this natural interface and suddenly everyone is a programmer because everyone speaks natural language like English. So this is extremely bullish and very interesting to me and also completely unprecedented. I would say it it used to be the case that you need to spend five to 10 years studying something to be able to do something in software. this is not the case anymore. So, I don't know if by any chance anyone has heard of vibe coding. Uh, this this is the tweet that kind of like introduced this, but I'm told that this is now like a major meme. Um, fun story about this is that I've been on Twitter for like 15 years\ngets $5 in credits for free when you sign up. And therefore, this is a major cost center in my life. So, this is a negative negative uh, revenue app for me right now. I've lost a huge amount of money 

In [97]:
question ="what is vibe coding?"

In [98]:
final_prompt=prompt.invoke({"context":content_text, "question":question})

In [99]:
print(final_prompt)

text="Answer ONLY from the provided transcript context.\n    If the context is insuffcient, just say you don't know .\nI think is very unique. Not only is there a new type of programming language that allows for autonomy in software but also as I mentioned it's programmed in English which is this natural interface and suddenly everyone is a programmer because everyone speaks natural language like English. So this is extremely bullish and very interesting to me and also completely unprecedented. I would say it it used to be the case that you need to spend five to 10 years studying something to be able to do something in software. this is not the case anymore. So, I don't know if by any chance anyone has heard of vibe coding. Uh, this this is the tweet that kind of like introduced this, but I'm told that this is now like a major meme. Um, fun story about this is that I've been on Twitter for like 15 years\ngets $5 in credits for free when you sign up. And therefore, this is a major cost 

In [100]:
answer=llm.invoke(final_prompt)
print(answer.content)

Vibe coding is a new type of programming language that allows for autonomy in software and is programmed in English, making it accessible to everyone who speaks natural language. It represents a shift in programming paradigms, enabling people to create software without needing extensive prior knowledge or years of study.


## Building a Chain

In [109]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [110]:
def format_docs(retrieved_docs):
    context_text="\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text



In [111]:
parallel_chain=RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question':RunnablePassthrough()
})

In [112]:
parallel_chain.invoke('what is augmentation and agents?')

{'context': "on can we just build for agents? I don't want to do this work. Can agents do this? Thank you. Okay. So roughly speaking, I think there's a new category of consumer and manipulator of digital information. It used to be just humans through GUIs or computers through APIs. And now we have a completely new thing and agents are they're computers but they are humanlike kind of right they're people spirits there's people spirits on the internet and they need to interact with our software infrastructure like can we build for them it's a new thing so as an example you can have robots.txt on your domain and you can instruct uh or like advise I suppose um uh web crawlers on how to behave on your website in the same way you can have maybe lm.txt txt file which is just a simple markdown that's telling\n\nmore kind of analogy that I always think through is the Iron Man suit. Uh I think this is I always love Iron Man. I think it's like so um correct in a bunch of ways with respect to tech

In [113]:
parser=StrOutputParser()


In [114]:
main_chain= parallel_chain | prompt |llm| parser

In [115]:
main_chain.invoke('what is augmentation and agents?')

"Augmentation refers to enhancements or improvements to existing capabilities, while agents are autonomous systems that can act on behalf of a user or perform tasks independently. In the context provided, the Iron Man suit serves as an analogy for both augmentation (enhancing Tony Stark's abilities) and agents (acting autonomously in some scenarios)."